В этой тетрадке представлен код для извлечения данных об управлении с помощью API cococo

In [ ]:
import pandas as pd

In [23]:
with open('verbs.txt', 'r', encoding='utf-8') as f:
    verbs = f.read()

In [24]:
verbs = verbs.split()

In [60]:
CAT_data = pd.read_csv('top1000_verbs_CAT.csv')
only_academic_top = [lemma for lemma in list(CAT_data['lemma']) if lemma not in verbs]

In [61]:
academic_verbs_data = pd.DataFrame({'verbs':only_academic_top})

In [25]:
verbs_data = pd.DataFrame({'verbs':verbs})

In [26]:
verbs_data

,verbs
0,быть
1,мочь
2,сказать
3,говорить
4,знать
...,...
996,разбудить
997,распространяться
998,удержаться
999,пострадать


In [64]:
import requests
from requests.exceptions import HTTPError
from numpy import median # for printing all fval ranks above the rank median
from decimal import Decimal # for printing freq ratio without truncation

In [63]:
#код Наму
def get_data(word):
    # configure default params to send to the server
    url = 'http://cococo.cosyco.ru/cococo-test.py'
    params = {
        'flimit': "10",
        'word1': word,        # Первое слово (ex. читать)
        'pos1': "",
        'features1': "",
        'word2': "",        # Второе слово (ex. сказать)
        'pos2': "",
        'features2': "",
        'corpus': "RNC",    # Используем ruscorpora
        'flem': "0",
        'alem': "0",
        'skip': "2"         # 2-граммы
    }

    # send get request to the server
    try:
        r = requests.get(url=url, params=params)

        # if server error, raise an exception
        r.raise_for_status()
    except HTTPError as http_err:
        print('HTTP error occurred:', http_err)
    except Exception as err:
        print('Other error occurred:', err)
    else:
        data = r.json()
        return data


In [56]:
data = get_data('читать')

In [57]:
data.keys()

dict_keys(['words', 'words_cl', 'poses', 'kl_lemma', 'klstr', 'kl_ordered'])

In [62]:
def get_cases_freq(word):
    data = get_data(word)
    if data:
        cases = {case: 0 for case in ['gen', 'dat', 'acc', 'ins', 'loc']}
        for pos in data['poses']:
            for feature in pos['features']:
                if feature['fname'] == 'case':
                    for case in feature['fvals']:
                        case_val = case['val']
                        if case_val in cases:
                            cases[case_val] += case['freqs'][0]
        return cases

In [42]:
verbs_data['cases'] = verbs_data['verbs'].apply(get_cases_freq)

In [66]:
academic_verbs_data['cases'] = academic_verbs_data['verbs'].apply(get_cases_freq)

In [6]:
def not_zero_key(features_dict):
    return {key:features_dict[key] for key in features_dict if features_dict[key]}

In [45]:
verbs_data['found_cases'] = verbs_data['cases'].apply(not_zero_key)

In [10]:
def with_thres(features_dict, tres=2):
    return {key:features_dict[key] for key in features_dict if features_dict[key] > tres}

In [68]:
academic_verbs_data['tres2'] = academic_verbs_data['cases'].apply(with_thres)

In [69]:
academic_verbs_data.to_csv('academic_verbs.csv')

In [2]:
import pandas as pd
pd.set_option("display.max_rows", 1001)

In [59]:
verbs_data

,verbs,cases,found_cases,tres2
0,быть,"{'gen': 97, 'dat': 48, 'acc': 41, 'ins': 997, 'loc': 0}","{'gen': 97, 'dat': 48, 'acc': 41, 'ins': 997}","{'gen': 97, 'dat': 48, 'acc': 41, 'ins': 997}"
1,мочь,"{'gen': 0, 'dat': 0, 'acc': 1, 'ins': 0, 'loc': 0}",{'acc': 1},{}
2,сказать,"{'gen': 38, 'dat': 192, 'acc': 121, 'ins': 17, 'loc': 7}","{'gen': 38, 'dat': 192, 'acc': 121, 'ins': 17, 'loc': 7}","{'gen': 38, 'dat': 192, 'acc': 121, 'ins': 17, 'loc': 7}"
3,говорить,"{'gen': 0, 'dat': 0, 'acc': 0, 'ins': 0, 'loc': 0}",{},{}
4,знать,"{'gen': 25, 'dat': 17, 'acc': 78, 'ins': 3, 'loc': 0}","{'gen': 25, 'dat': 17, 'acc': 78, 'ins': 3}","{'gen': 25, 'dat': 17, 'acc': 78, 'ins': 3}"
5,стать,"{'gen': 1, 'dat': 6, 'acc': 0, 'ins': 355, 'loc': 0}","{'gen': 1, 'dat': 6, 'ins': 355}","{'dat': 6, 'ins': 355}"
6,хотеть,"{'gen': 2, 'dat': 0, 'acc': 1, 'ins': 0, 'loc': 0}","{'gen': 2, 'acc': 1}",{}
7,идти,"{'gen': 0, 'dat': 0, 'acc': 0, 'ins': 0, 'loc': 0}",{},{}
8,иметь,"{'gen': 42, 'dat': 2, 'acc': 295, 'ins': 5, 'loc': 0}","{'gen': 42, 'dat': 2, 'acc': 295, 'ins': 5}","{'gen': 42, 'acc': 295, 'ins': 5}"
9,видеть,"{'gen': 15, 'dat': 1, 'acc': 197, 'ins': 2, 'loc': 0}","{'gen': 15, 'dat': 1, 'acc': 197, 'ins': 2}","{'gen': 15, 'acc': 197}"


In [8]:
academic_verbs = pd.read_csv('academic_verbs.csv')
academic_verbs

,Unnamed: 0,verbs,cases,tres2
0,0,нет,"{'gen': 1087, 'dat': 58, 'acc': 64, 'ins': 6, ...","{'gen': 1087, 'dat': 58, 'acc': 64, 'ins': 6}"
1,1,Выделить,"{'gen': 0, 'dat': 0, 'acc': 0, 'ins': 0, 'loc'...",{}
2,2,анализировать,"{'gen': 0, 'dat': 0, 'acc': 13, 'ins': 1, 'loc...",{'acc': 13}
3,3,фиксировать,"{'gen': 0, 'dat': 0, 'acc': 8, 'ins': 0, 'loc'...",{'acc': 8}
4,4,сровнять,"{'gen': 0, 'dat': 0, 'acc': 0, 'ins': 0, 'loc'...",{}
5,5,характеризовать,"{'gen': 0, 'dat': 0, 'acc': 4, 'ins': 0, 'loc'...",{'acc': 4}
6,6,формулировать,"{'gen': 1, 'dat': 0, 'acc': 5, 'ins': 0, 'loc'...",{'acc': 5}
7,7,употреблять,"{'gen': 5, 'dat': 0, 'acc': 15, 'ins': 0, 'loc...","{'gen': 5, 'acc': 15}"
8,8,обучаться,"{'gen': 0, 'dat': 0, 'acc': 0, 'ins': 0, 'loc'...",{}
9,9,ориентировать,"{'gen': 0, 'dat': 0, 'acc': 0, 'ins': 0, 'loc'...",{}


In [15]:
def get_data(word):
    # configure default params to send to the server
    url = 'http://cococo.cosyco.ru/cococo-test.py'
    params = {
        'flimit': "",
        'word1': word,        # Первое слово (ex. читать)
        'pos1': "",
        'features1': "",
        'word2': "",        # Второе слово (ex. сказать)
        'pos2': 'PR',
        'features2': "",
        'corpus': "RNC",    # Используем ruscorpora
        'flem': "0",
        'alem': "0",
        'skip': "2"         # 3-граммы
    }

    # send get request to the server
    try:
        r = requests.get(url=url, params=params)

        # if server error, raise an exception
        r.raise_for_status()
    except HTTPError as http_err:
        print('HTTP error occurred:', http_err)
    except Exception as err:
        print('Other error occurred:', err)
    else:
        data = r.json()
        return data

In [21]:
def get_preposition_data(verb):
    words_data = get_data(verb)['words']
    if words_data:
        return {word['word']: word['freq1'] for word in words_data}

In [28]:
verbs_data

,verbs,PRs
0,быть,"{'в': 212, 'с': 70, 'на': 63, 'у': 57, 'для': ..."
1,мочь,None
2,сказать,"{'в': 46, 'на': 26, 'по': 20, 'об': 16, 'с': 1..."
3,говорить,"{'о': 264, 'с': 84, 'об': 62, 'в': 22, 'на': 1..."
4,знать,"{'о': 35, 'про': 13, 'в': 13, 'об': 5, 'с': 5,..."
...,...,...
996,разбудить,"{'среди': 1, 'во': 1, 'от': 1, 'по': 1}"
997,распространяться,"{'о': 5, 'на': 3, 'в': 3, 'насчeт': 2, 'во': 2..."
998,удержаться,"{'от': 15, 'на': 6, 'в': 4, 'у': 2, 'между': 1..."
999,пострадать,"{'за': 2, 'при': 1}"


In [34]:
case_options_preps = ['в', 'над', 'за', 'под', 'c', 'по']

In [29]:
def get_data_3grams(word1, word2):
    # configure default params to send to the server
    url = 'http://cococo.cosyco.ru/cococo-test.py'
    params = {
        'flimit': "",
        'word1': word1,        # Первое слово (ex. читать)
        'pos1': "",
        'features1': "",
        'word2': word2,        # Второе слово (ex. сказать)
        'pos2': '',
        'features2': "",
        'word3': '', 
        'pos3': "",
        'features3': "",
        'corpus': "RNC",    # Используем ruscorpora
        'flem': "0",
        'alem': "0",
        'skip': "3"         # 3-граммы
    }

    # send get request to the server
    try:
        r = requests.get(url=url, params=params)

        # if server error, raise an exception
        r.raise_for_status()
    except HTTPError as http_err:
        print('HTTP error occurred:', http_err)
    except Exception as err:
        print('Other error occurred:', err)
    else:
        data = r.json()
        return data

In [30]:
data = get_data_3grams('читать', 'о')

In [33]:
data['poses']

[{'posname': 'A',
  'freqs': [1, 445839, 2.24e-06],
  'kl': 0.0,
  'kln': 0.0,
  'features': [{'fname': 'abbr',
    'kl': 0.0,
    'fvals': [{'val': 'nonabbr', 'freqs': [1, 444850, 2.25e-06]}]},
   {'fname': 'animacy',
    'kl': 0.0,
    'fvals': [{'val': 'anim', 'freqs': [0, 4679, 0.0]},
     {'val': 'inan', 'freqs': [0, 48200, 0.0]}]},
   {'fname': 'case',
    'kl': 2.4,
    'fvals': [{'val': 'acc', 'freqs': [0, 73043, 0.0]},
     {'val': 'dat', 'freqs': [0, 13802, 0.0]},
     {'val': 'gen', 'freqs': [0, 103494, 0.0]},
     {'val': 'ins', 'freqs': [0, 52241, 0.0]},
     {'val': 'loc', 'freqs': [1, 36553, 2.74e-05]},
     {'val': 'nom', 'freqs': [0, 125533, 0.0]}]},
   {'fname': 'degree',
    'kl': 0.0,
    'fvals': [{'val': 'comp', 'freqs': [0, 4414, 0.0]},
     {'val': 'comp2', 'freqs': [0, 466, 0.0]}]},
   {'fname': 'form',
    'kl': 0.0,
    'fvals': [{'val': 'plen', 'freqs': [1, 0, 0.0]}]},
   {'fname': 'gender',
    'kl': 0.0,
    'fvals': [{'val': 'f', 'freqs': [0, 115696, 0.0]

In [35]:
def get_cases_freq(data):
    if data:
        cases = {case: 0 for case in ['gen', 'dat', 'acc', 'ins', 'loc']}
        for pos in data['poses']:
            for feature in pos['features']:
                if feature['fname'] == 'case':
                    for case in feature['fvals']:
                        case_val = case['val']
                        if case_val in cases:
                            cases[case_val] += case['freqs'][0]
        return cases

In [45]:
def choose_case_after_prep(verb, prep_dict):
    if prep_dict:
        found_cases = {}
        for prep in prep_dict:
            if prep in case_options_preps:
                data = get_data_3grams(verb, prep)
                found_cases[prep] = get_cases_freq(data)
        return found_cases

In [46]:
verbs_data['cases_after_preps'] = verbs_data.apply(lambda x: choose_case_after_prep(x.verbs, x.PRs), axis=1)

In [47]:
verbs_data

,verbs,PRs,cases_after_preps
0,быть,"{'в': 212, 'с': 70, 'на': 63, 'у': 57, 'для': ...","{'в': {'gen': 1, 'dat': 0, 'acc': 33, 'ins': 0..."
1,мочь,None,None
2,сказать,"{'в': 46, 'на': 26, 'по': 20, 'об': 16, 'с': 1...","{'в': {'gen': 0, 'dat': 0, 'acc': 16, 'ins': 0..."
3,говорить,"{'о': 264, 'с': 84, 'об': 62, 'в': 22, 'на': 1...","{'в': {'gen': 0, 'dat': 0, 'acc': 11, 'ins': 0..."
4,знать,"{'о': 35, 'про': 13, 'в': 13, 'об': 5, 'с': 5,...","{'в': {'gen': 0, 'dat': 0, 'acc': 6, 'ins': 0,..."
...,...,...,...
996,разбудить,"{'среди': 1, 'во': 1, 'от': 1, 'по': 1}","{'по': {'gen': 0, 'dat': 1, 'acc': 0, 'ins': 0..."
997,распространяться,"{'о': 5, 'на': 3, 'в': 3, 'насчeт': 2, 'во': 2...","{'в': {'gen': 0, 'dat': 0, 'acc': 1, 'ins': 0,..."
998,удержаться,"{'от': 15, 'на': 6, 'в': 4, 'у': 2, 'между': 1...","{'в': {'gen': 0, 'dat': 0, 'acc': 0, 'ins': 0,..."
999,пострадать,"{'за': 2, 'при': 1}","{'за': {'gen': 0, 'dat': 0, 'acc': 2, 'ins': 0..."


In [48]:
verbs_data.to_csv('prepositions_after_verbs.csv')